# Assignment 1: Data Description
CS 6530
Dillon Daudert

## Question 1
From the file `data.online.txt`, calculate the following statistics about the midterm scores and final scores:
1. max, min
2. quartiles
3. mean
4. mode
5. variance

In [1]:
# load the data, tab separated: student_id | mid-term_score | final_score
using CSV, DataFrames, StatsBase
data = CSV.read("./A1/data.online.scores.txt"; delim='\t', header=["id", "midterm", "final"], allowmissing=:none)
head(data)

INFO: Recompiling stale cache file /home/dillon/.julia/lib/v0.6/CSV.ji for module CSV.


,id,midterm,final
1,0,97,95
2,1,83,99
3,2,79,81
4,3,72,84
5,4,92,84
6,5,78,91


In [2]:
print("Midterm scores:\n")
describe(data[:midterm])
@printf "%-15s %3.6f\n" "Mode:" mode(data[:midterm])
@printf "%-15s %3.6f\n" "Variance:" var(data[:midterm])

Midterm scores:
Summary Stats:
Mean:           76.715000
Minimum:        37.000000
1st Quartile:   68.000000
Median:         77.000000
3rd Quartile:   87.000000
Maximum:        100.000000
Length:         1000
Type:           Int64
Mode:           77.000000
Variance:       173.279054


In [3]:
print("Final scores:\n")
describe(data[:final])
@printf "%-15s %3.6f\n" "Mode:" mode(data[:final])
@printf "%-15s %3.6f\n" "Variance:" var(data[:midterm])

Final scores:
Summary Stats:
Mean:           87.084000
Minimum:        35.000000
1st Quartile:   82.000000
Median:         89.000000
3rd Quartile:   96.000000
Maximum:        100.000000
Length:         1000
Type:           Int64
Mode:           97.000000
Variance:       173.279054


## Question 2
Use the data from the same file, normalize the mid-term score using z-score normalization.
1. Compare the variance before and after normalization
2. Given an initial score of 90, what is the corresponding score after normalization?
3. What is the Pearson's correlation coefficient between midterm scores and final scores?
4. What is the covariance between midterm scores and final scores?

In [4]:
normalized_midterm = (data[:midterm] .- mean(data[:midterm]))./std(data[:midterm])
@printf "Midterm variance: %3.2f, after normalization: %3.2f" var(data[:midterm]) var(normalized_midterm)

Midterm variance: 173.28, after normalization: 1.00

In [5]:
@printf "Initial score of 90 after normalization: %3.2f" (90. - mean(data[:midterm])) / std(data[:midterm])

Initial score of 90 after normalization: 1.01

In [6]:
print("Pearson's correlation coefficient between midterm and final scores:")
cor([data[:midterm] data[:final]])

Pearson's correlation coefficient between midterm and final scores:

2×2 Array{Float64,2}:
 1.0       0.544425
 0.544425  1.0     

In [7]:
print("Covariance between midterm and final scores:")
cov([data[:midterm] data[:final]])

Covariance between midterm and final scores:

2×2 Array{Float64,2}:
 173.279    78.2542
  78.2542  119.232 

## Question 3
Given the inventories of two libraries, compare the similarity between the two libraries by using different proximity measures.
- Using 200 books, Table 1 summarizes how many books are supplied by corresponding library. For example, the table cell for CBL=0 and CML=0 is 20, which means there are 20 books that are served neither by CBL nor CML. The table cell for CBL=1 and CML=0 is 2, which means that there are 2 books that are served by CBL but not CML. Using Table 1, calculate the Jaccard coefficient of CBL and CML.

In [8]:
using Distances

INFO: Precompiling module Distances.


In [9]:
# Jaccard similarity coefficient of two sets is defined as the size of the intersection divided by the size of the union
intersection = 58
union = 120 + 2 + 58
jaccard = intersection / union
@printf "Jaccard coefficient: %.3f" jaccard

Jaccard coefficient: 0.322

- File `data.libraries.inventories.txt` has two data records, on record for each library. Each record shows # of copies of 100 selected books the library owns. Calculate the Minkowski distance of the two records with regard to different h values.
  1. h = 1
  2. h = 2
  3. h = `Inf`

In [32]:
inv_data = CSV.read("./A1/data.libraries.inventories.txt", delim='\t', allowmissing=:none)
head(inv_data)

,library,book_1,book_2,book_3,book_4,book_5,book_6,book_7,book_8,book_9,book_10,book_11,book_12,book_13,book_14,book_15,book_16,book_17,book_18,book_19,book_20,book_21,book_22,book_23,book_24,book_25,book_26,book_27,book_28,book_29,book_30,book_31,book_32,book_33,book_34,book_35,book_36,book_37,book_38,book_39,book_40,book_41,book_42,book_43,book_44,book_45,book_46,book_47,book_48,book_49,book_50,book_51,book_52,book_53,book_54,book_55,book_56,book_57,book_58,book_59,book_60,book_61,book_62,book_63,book_64,book_65,book_66,book_67,book_68,book_69,book_70,book_71,book_72,book_73,book_74,book_75,book_76,book_77,book_78,book_79,book_80,book_81,book_82,book_83,book_84,book_85,book_86,book_87,book_88,book_89,book_90,book_91,book_92,book_93,book_94,book_95,book_96,book_97,book_98,book_99,book_100
1,CML,53,136,90,103,36,93,195,51,192,148,79,30,48,80,242,77,170,137,41,146,116,186,101,173,110,50,149,139,30,119,86,97,25,63,173,130,60,148,129,69,167,83,121,28,133,173,149,105,64,134,137,21,179,116,35,119,60,140,150,68,145,197,80,73,128,3,180,96,111,95,122,132,64,181,165,129,79,63,72,187,99,143,49,131,84,179,215,112,76,64,50,144,46,110,163,183,107,129,162,173
2,CBL,103,80,132,167,144,172,108,89,103,63,65,58,141,116,152,122,129,83,175,116,173,67,61,114,195,166,85,42,87,183,217,125,109,71,93,71,77,209,134,59,99,147,75,50,181,102,173,160,147,97,90,57,151,100,102,181,118,163,141,146,149,149,250,128,155,148,123,189,80,186,80,114,102,31,66,34,59,213,104,251,31,75,164,214,53,79,118,75,169,192,92,139,122,123,123,61,135,90,125,118


In [28]:
# h = 1
h₁ = minkowski(Array(inv_data[1, 2:end]), Array(inv_data[2, 2:end]), 1)
@printf "Minkowski h₁: %3.3f\n" h₁
# h = 2
h₂ = minkowski(Array(inv_data[1, 2:end]), Array(inv_data[2, 2:end]), 2)
@printf "Minkowski h₂: %3.3f\n" h₂
# h = ∞
h₃ = chebyshev(Array(inv_data[1, 2:end]), Array(inv_data[2, 2:end]))
@printf "Minkowski h=∞: %3.3f\n" h₃

Minkowski h₁: 6152.000
Minkowski h₂: 715.328
Minkowski h=∞: 170.000


- Calculate the cosine similary between the two libraries

In [29]:
cos_d = cosine_dist(Array(inv_data[1, 2:end]), Array(inv_data[2, 2:end]))
@printf "Cosine distance: %3.3f" cos_d

Cosine distance: 0.159

- Calculate the KL divergence between the two libraries. Assume a person picks up a book in the library randomly. The probability that they pick up book i is therefore bᵢ/(sumₖ(bₖ))

In [34]:
cml_prob = Array(inv_data[1, 2:end]) ./ sum(Array(inv_data[1, 2:end]))
cml_prob

1×100 Array{Float64,2}:
 0.00473088  0.0121396  0.00803356  …  0.0115148  0.0144604  0.0154423

In [36]:
cbl_prob = Array(inv_data[2, 2:end]) ./ sum(Array(inv_data[2, 2:end]))
cbl_prob

1×100 Array{Float64,2}:
 0.00855127  0.00664176  0.0109589  …  0.00747198  0.0103778  0.0097966

In [39]:
kl_d₁ = kl_divergence(cml_prob, cbl_prob)
kl_d₂ = kl_divergence(cbl_prob, cml_prob)
@printf "KL Divergence of CML and CBL: %3.3f\n" kl_d₁
@printf "KL Divergence of CBL and CML: %3.3f\n" kl_d₂


KL Divergence of CML and CBL: 0.207
KL Divergence of CBL and CML: 0.230


## Question 4
Using Table 2, calculate the chi-squared correlation value.